<a href="https://colab.research.google.com/github/sujay-dsa/NLP/blob/main/Assignment_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)

    self.fc3 = nn.Linear(in_features = 20, out_features = 30) # 20 inputs because 10 inputs from results of image prediction and 10 from random number encoded
    self.out_final = nn.Linear(in_features=30, out_features=20) 

  def forward(self, t, intTensor):
    # input layer
    x = t
    
    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12
    
    
    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)

    # fc1 layer
    x = self.fc1(x)
    x = F.relu(x)

    # fc2 layer
    x = self.fc2(x)
    x = F.relu(x)

    # output layer of MNIST
    x = self.out(x)
    x = F.softmax(x, dim=1)
    img_pred = x.clone()
    
    
    
    # fc3 layer
    x = torch.cat((x, intTensor),1) # combine the random number tensor with the result of image prediction
    x = self.fc3(x)
    x = F.relu(x)

    x = self.out_final(x) # this is the sum
    x = F.softmax(x, dim=1)
    # x = torch.cat((x, img_pred),1)
    
    return img_pred, x

In [ ]:
torch.set_grad_enabled(False)

In [ ]:
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform = transforms.Compose([
          transforms.ToTensor()
    ])
)

In [ ]:
randomNumberTensor = torch.randint(0,9, size=(60000,))
randomNumberTensor,train_set.targets.shape

(tensor([6, 5, 1,  ..., 7, 6, 7]), torch.Size([60000]))

In [ ]:
bkup = train_set.targets.clone()
train_set.targets, randomNumberTensor

(tensor([9, 0, 0,  ..., 3, 0, 5]), tensor([6, 5, 1,  ..., 7, 6, 7]))

In [ ]:
final_targets = train_set.targets + randomNumberTensor
final_targets

tensor([15,  5,  1,  ..., 10,  6, 12])

In [ ]:
randomOHE = F.one_hot(randomNumberTensor, num_classes=10)
randomTargetsOHE = F.one_hot(final_targets, num_classes=20)
randomNumberTensor[0],randomOHE[0]

(tensor(6), tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]))

In [ ]:
from torch.utils.data import Dataset

class myDataSet(Dataset):
  def __init__(self, inputs, labels):
    self.data = F.one_hot(inputs, num_classes=10)
    self.targets = labels

  def __getitem__(self, index):
    encoding = self.data[index]
    label = self.targets[index]
    return encoding, label.item()

  def __len__(self):
    return len(self.data)

In [ ]:
myDS = myDataSet(randomNumberTensor, final_targets)
x = iter(myDS)
encoding, label = next(x)
encoding.shape,encoding.unsqueeze(0).shape, label

(torch.Size([10]), torch.Size([1, 10]), 15)

In [ ]:
sample = next(iter(train_set)) 
image, label = sample
image.shape, image.unsqueeze(0).shape, label


(torch.Size([1, 28, 28]), torch.Size([1, 1, 28, 28]), 9)

In [ ]:
network = Network() 
pred = network(image.unsqueeze(0), encoding.unsqueeze(0))

In [ ]:
pred, pred[0].shape, label

((tensor([[0.0905, 0.1009, 0.0963, 0.1110, 0.1101, 0.1105, 0.0961, 0.0941, 0.0997,
           0.0908]]),
  tensor([[0.0512, 0.0508, 0.0477, 0.0642, 0.0546, 0.0448, 0.0475, 0.0537, 0.0561,
           0.0480, 0.0525, 0.0509, 0.0599, 0.0401, 0.0470, 0.0435, 0.0514, 0.0412,
           0.0484, 0.0464]])),
 torch.Size([1, 10]),
 9)

In [ ]:
pred[0].argmax(dim=1),pred[1].argmax(dim=1), train_set.targets, randomNumberTensor

(tensor([3]),
 tensor([3]),
 tensor([9, 0, 0,  ..., 3, 0, 5]),
 tensor([6, 5, 1,  ..., 7, 6, 7]))

In [ ]:
F.softmax(pred[0], dim=1)

tensor([[0.0991, 0.1001, 0.0996, 0.1011, 0.1010, 0.1011, 0.0996, 0.0994, 0.1000,
         0.0991]])

In [ ]:
F.softmax(pred[0], dim=1).sum()

tensor(1.)

In [ ]:
myDS, train_set

(<__main__.myDataSet at 0x7f9659a6fd50>, Dataset FashionMNIST
     Number of datapoints: 60000
     Root location: ./data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
            ))

In [ ]:
train_set.data.shape, train_set.targets.shape

(torch.Size([60000, 28, 28]), torch.Size([60000]))

In [ ]:
temp = train_set.targets.clone()
temp = torch.stack((temp, myDS.targets),dim=1)

In [ ]:
temp.shape, train_set.targets[:10], randomNumberTensor[:10], temp[:10]

(torch.Size([60000, 2]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]),
 tensor([6, 5, 1, 6, 0, 1, 1, 6, 4, 8]),
 tensor([[ 9, 15],
         [ 0,  5],
         [ 0,  1],
         [ 3,  9],
         [ 0,  0],
         [ 2,  3],
         [ 7,  8],
         [ 2,  8],
         [ 5,  9],
         [ 5, 13]]))

In [ ]:
myDS.data[0,:], train_set.data[0].shape, train_set.targets[0], myDS.data[0], myDS.targets[0]

(tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 torch.Size([28, 28]),
 tensor(9),
 tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 tensor(15))

In [ ]:
class myCombinedDataSet(Dataset):
  def __init__(self, imageDataSet, integerDataSet, transform=None):
    self.imageInputs = imageDataSet.data
    self.integerInputs = integerDataSet.data
    self.imageDataSet = imageDataSet
    self.integerDataSet = integerDataSet
    self.transform = transform
    print(len(self.imageInputs))

  def __getitem__(self, index):
    imageInput = self.imageInputs[index]
    integerInput = self.integerInputs[index]
    imageLabel = self.imageDataSet.targets[index]
    sumLabel = self.integerDataSet.targets[index]
    sample = {'imageInput':imageInput, 'integerInput': integerInput, 'imageLabel': imageLabel, 'sumLabel': sumLabel}
    return sample

  def __len__(self):
    print(len(self.imageInputs))
    return len(self.imageInputs)

myCombinedDS = myCombinedDataSet(train_set, myDS)

60000


In [ ]:
data_loader = torch.utils.data.DataLoader(
    myCombinedDS, 
    batch_size=10

)

train_set.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [ ]:
batch = next(iter(data_loader))
batch['integerInput'], batch['sumLabel']

60000


(tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]),
 tensor([15,  5,  1,  9,  0,  3,  8,  8,  9, 13]))

In [ ]:
batch['imageInput'].shape, batch['integerInput'].shape

(torch.Size([10, 28, 28]), torch.Size([10, 10]))

In [ ]:
batch['imageInput'].unsqueeze(1).shape, batch['integerInput'][0]

(torch.Size([10, 1, 28, 28]), tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]))

In [ ]:
temp_data_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=10
)
batch1 = next(iter(temp_data_loader))

In [ ]:
images, labels = batch1
images.shape, labels

(torch.Size([10, 1, 28, 28]), tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))

In [ ]:
network = Network() 
pred = network(image.unsqueeze(0), encoding.unsqueeze(0))

In [ ]:
batch['integerInput'].shape,encoding.unsqueeze(0).shape,batch['integerInput']

(torch.Size([10, 10]),
 torch.Size([1, 10]),
 tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]))

In [ ]:
torch.rand((10,28,28)).unsqueeze(1).shape,batch['imageInput'].unsqueeze(1).shape,  batch['integerInput'].reshape(10,-1).shape

(torch.Size([10, 1, 28, 28]),
 torch.Size([10, 1, 28, 28]),
 torch.Size([10, 10]))

In [ ]:
network = Network() 
preds = network(torch.rand((10,28,28)).unsqueeze(1), batch['integerInput'].reshape(10,-1))
batch['imageInput'].shape, batch['imageInput'].float().dtype, torch.rand((10,28,28)).unsqueeze(1).dtype

(torch.Size([10, 28, 28]), torch.float32, torch.float32)

In [ ]:
network = Network() 
preds = network(batch['imageInput'].unsqueeze(1).float(), batch['integerInput'].reshape(10,-1).float())
preds[0].shape, preds[1].shape

(torch.Size([10, 10]), torch.Size([10, 20]))

In [ ]:
preds[0].argmax(dim=1), preds[1].argmax(dim=1)

(tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 tensor([15, 15, 15, 15, 15, 15, 15, 15, 15, 15]))

In [ ]:
preds[0].argmax(dim=1).eq(batch['imageLabel']), preds[1].argmax(dim=1).eq(batch['sumLabel'])

(tensor([False, False, False,  True, False, False, False, False, False, False]),
 tensor([ True, False, False, False, False, False, False, False, False, False]))

In [ ]:
def get_num_correct(preds, imageLabels, sumLabels):
  return preds[0].argmax(dim=1).eq(imageLabels), preds[1].argmax(dim=1).eq(sumLabels)

In [ ]:
get_num_correct(preds, preds[0].argmax(dim=1).eq(batch['imageLabel']), preds[1].argmax(dim=1).eq(batch['sumLabel']))

(tensor([False, False, False, False, False, False, False, False, False, False]),
 tensor([False, False, False, False, False, False, False, False, False, False]))

In [ ]:
import torch.optim as optim

torch.set_grad_enabled(True)

In [ ]:
preds = network(batch['imageInput'].unsqueeze(1).float(), batch['integerInput'].reshape(10,-1).float())
loss_images = F.cross_entropy(preds[0], batch['imageLabel'])
loss_sum = F.cross_entropy(preds[1], batch['sumLabel'])
total_loss = loss_images + loss_sum
total_loss.item()

5.345405578613281

In [ ]:
pred[0].shape, batch['imageLabel'], pred[1].shape, batch['sumLabel']

(torch.Size([1, 10]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]),
 torch.Size([1, 20]),
 tensor([15,  5,  1,  9,  0,  3,  8,  8,  9, 13]))

In [ ]:
total_loss.backward(retain_graph=True)

In [ ]:
print(network.conv1.weight.grad.shape)

torch.Size([6, 1, 5, 5])


In [ ]:
optimizer = optim.Adam(network.parameters(), lr = 0.01)

In [ ]:
total_loss.item(), get_num_correct(preds, batch['imageLabel'], batch['sumLabel'])

(5.345405578613281,
 (tensor([False, False, False,  True, False, False, False, False, False, False]),
  tensor([ True, False, False, False, False, False, False, False, False, False])))

In [ ]:
optimizer.step()

In [ ]:
total_loss.backward(retain_graph=True)
preds = network(batch['imageInput'].unsqueeze(1).float(), batch['integerInput'].reshape(10,-1).float())
total_loss.item()

RuntimeError: ignored

In [ ]:
total_loss.backward(retain_graph=True)
preds = network(batch['imageInput'].unsqueeze(1).float(), batch['integerInput'].reshape(10,-1).float())
total_loss.item()

In [ ]:
batch['sumLabel']

In [ ]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

batch = next(iter(train_loader)) # Get Batch
images, labels = batch

ohe_batch_100 = randomOHE[:100]
preds = network(images,ohe_batch_100) # Pass Batch
loss = F.cross_entropy(preds, labels) # Calculate Loss

loss.backward() # Calculate Gradients
optimizer.step() # Update Weights

print('loss1:', loss.item())
preds = network(images,ohe_batch_100)
loss = F.cross_entropy(preds, labels)
print('loss2:', loss.item())

In [ ]:
torch.set_grad_enabled(False)
torch.set_grad_enabled(True)

In [ ]:
randomOHE[0:100,:].shape

In [ ]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

total_loss = 0
total_correct = 0
ohe_batch = 0

for idx, batch in enumerate(train_loader): # Get Batch
    images, labels = batch 
    
    
    preds = network(images, randomOHE[ohe_batch: (idx+1)*100,:]) # Pass Batch
    loss = F.cross_entropy(preds, labels) # Calculate Loss

    optimizer.zero_grad()
    loss.backward() # Calculate Gradients
    optimizer.step() # Update Weights

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

    ohe_batch = 100*(idx + 1)

print(
    "epoch:", 0, 
    "total_correct:", total_correct, 
    "loss:", total_loss
)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):

    total_loss = 0
    total_correct = 0
    ohe_batch = 0


    for idx, batch in enumerate(train_loader): # Get Batch
        images, labels = batch 

        preds = network(images, randomOHE[ohe_batch: (idx+1)*100,:]) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

        ohe_batch = 100*(idx + 1)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

Conv2 >> 12 * 4 * 4
Conv2.reshape >> 192
random_number input >> 10
fc_rn (10, 20) >> 20
concat_data = concatenation (192, 20)
combined_data >> 212
fc2(212) >> 100 << fc2_output
Approach 1
fc3(100) >> 10 + 19 OR 10 + 5 << out
fc3(100) >> 19 OR 15 << out
Approach 2
fc3(fc2_output) >> 10 << MNIST VALUE
fc4(fc2_output) >> 19/5 << SUM VALUE
return MNIST_VALUE, SUM_VALUE

return out

loss_for_mnist out[:10]
loss_for_sum out[10:]